In [1]:
import ee
import pandas as pd
import asset_paths_to_cop_data as fc_paths
import make_difference_images as di_module

ee.Initialize()

In [47]:
def import_FCs():
    subsite_filenames = [y for x in fc_paths.ALL_FILENAMES for y in x]
    poly_data = []
    for ss_fc_tuple in subsite_filenames:
        ss_name = ss_fc_tuple[0].split("/")[3]
        asset_id = ss_fc_tuple[0].split("/")[4]
        roi = ee.FeatureCollection(ss_fc_tuple[0])
        flood_poly = ee.FeatureCollection(ss_fc_tuple[1])
        poly_data.append([ss_name, asset_id, roi, flood_poly])
    return poly_data

In [42]:
def get_S1_images(all_geometries):
    return ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
        .filterBounds(all_geometries) \
        .select(['VV'])

In [54]:
def create_labelled_dataset(poly_and_di_df):
    labelled_data = []
    for i, row in poly_and_di_df.iterrows():
        geom, flood, di = row['Geometry'], row['Flood Poly'], row['Difference Image']
        bounds = ee.FeatureCollection(geom.geometry().bounds())
        empty = ee.Image.constant(0).byte().clip(bounds)
        painted = empty.paint(flood, 1)
        di_with_flood = di.addBands(painted.clip(bounds)).addBands(ee.Image.pixelLonLat())
#         training = di_with_flood.stratifiedSample(10000,'constant', bounds,10,'EPSG:4326', geometries=True)
        labelled_data.append([row['Subsites'], di_with_flood])
    final_df = poly_and_di_df.merge(pd.DataFrame(labelled_data, columns=['Subsites', 'Labelled Data']))
    return final_df

In [107]:
def batch_export_images_as_tfrecords_to_GCS(image, bounds, file_name_prefix):
  task = ee.batch.Export.image.toCloudStorage(
    image=image,
    bucket='labelled_data',
    fileNamePrefix=file_name_prefix,
    region=bounds.getInfo()['coordinates'],
    scale=10,
    maxPixels=1E10,
    fileFormat='TFRecord',
    formatOptions={'patchDimensions':[256,256]}
  )
  task.start()

In [57]:
poly_data = import_FCs()
poly_df = pd.DataFrame(poly_data, columns=['Subsites', 'Asset Id', 'Geometry', 'Flood Poly'])
all_geometries = ee.FeatureCollection(poly_df['Geometry'].tolist()).flatten()

In [58]:
# poly_df[0:11]

In [59]:
images = get_S1_images(all_geometries)
poly_and_di_df = di_module.add_DIs(poly_df, images)

In [60]:
final_df = create_labelled_dataset(poly_and_di_df)

In [108]:
for i, row in final_df.iterrows():
    image_to_export = row['Labelled Data']
    batch_export_images_as_tfrecords_to_GCS(image_to_export, row['Geometry'].geometry().bounds(), row['Asset Id'])